In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV,cross_val_score
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score


In [2]:
selected_features=['partner',
 'onlinebackup',
 'tenure',
 'contract',
 'monthlycharges',
 'totalcharges',
 'techsupport', 'internetservice',
 'multiplelines']

In [3]:
x_train_bal=pd.read_csv("x_train_bal.csv")
y_train_bal=pd.read_csv("y_train_bal.csv")
x_train_bal=x_train_bal[selected_features]

In [5]:
x_train_bal.head()

,partner,onlinebackup,tenure,contract,monthlycharges,totalcharges,techsupport,internetservice,multiplelines
0,1,1,65,2,94.55,6078.75,1,2,1
1,0,0,26,0,35.75,1022.50,1,1,2
2,1,1,68,2,90.20,6297.65,1,2,1
3,0,1,3,0,84.30,235.05,0,2,0
4,1,0,49,0,40.65,2070.75,0,1,2


In [8]:
y_train_bal.value_counts()

churn
0        4130
1        4130
Name: count, dtype: int64

In [9]:
param_grid ={
    'n_estimators': [50, 100, 200],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20],
    'max_features': ['sqrt', 'log2'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True],
    'max_leaf_nodes': [None, 10, 20],
    'min_impurity_decrease': [0.0, 0.01],
    'class_weight': ['balanced'],
    'warm_start': [False, True],
    'verbose': [0]
}

In [10]:
grid_skf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    cv=5,               # <-- use StratifiedKFold here
    scoring='accuracy',
    verbose=3
)

In [ ]:
grid_skf.fit(x_train_bal, y_train_bal)
print("Best params (StratifiedKFold):", grid_skf.best_params_)